In [27]:
using CSV, DataFrames, Statistics, FreqTables, LinearAlgebra

In [8]:
a1=DataFrame(CSV.File("./csv/asts.csv"))

,ast,courseid,astname,cont
,String7,Int64,String,Float64
1,dqfe,675,Lab Report 1,0.0175
2,eqqr,104,Lab Report 1,0.0161538
3,purz,104,Lab Report 2,0.0161538
4,zteo,675,Lab Report 2,0.0175
5,knny,832,Essay 1,0.25
6,qrfq,675,Lab Report 3,0.0175
7,qdhd,104,Lab Report 3,0.0161538
8,svah,675,Lab Report 4,0.0175
9,pbif,104,Lab Report 4,0.0161538


In [9]:
m1=DataFrame(CSV.File("./csv/marks.csv"))

,ast,studentID,mark
,String7,Int64,Float64
1,dqfe,369773,6.67
2,dqfe,542150,80.0
3,dqfe,373971,80.0
4,dqfe,658468,26.67
5,dqfe,946387,73.33
6,dqfe,505926,48.87
7,dqfe,127232,0.0
8,dqfe,911293,100.0
9,dqfe,377866,80.0


In [10]:
a2=a1[a1.courseid .== 326,:] # filter for course id 326

,ast,courseid,astname,cont
,String7,Int64,String,Float64
1,mnll,326,Essay 1,0.25
2,nvxk,326,Final Exam,0.25
3,bemh,326,Final Essay,0.25
4,tcbq,326,Attendance,0.05
5,zijh,326,Reading Assessment Quizzes,0.2


In [18]:
d1=Dict{String, Float64}() # associate ast with cont
for i in 1:length(a2[:,:ast])
    merge!(d1,Dict(a2[i,:ast]=>a2[i,:cont]))
end

In [13]:
m2=DataFrame() # filter for marks relevant to course id 326
dup=[]
for i in 1:length(a2[:,:ast])
    m2=vcat(m2,m1[m1.ast .== a2[i,:ast],:])
    dup=vcat(dup,sum(nonunique(m1[m1.ast .== a2[i,:ast],:],:studentID)))
end

In [22]:
m2

,ast,studentID,mark
,String7,Int64,Float64
1,mnll,104689,89.0
2,mnll,114185,92.0
3,mnll,116679,90.0
4,mnll,116824,89.0
5,mnll,162638,89.0
6,mnll,205269,92.0
7,mnll,224062,89.0
8,mnll,251559,90.0
9,mnll,252873,90.0


In [14]:
dup # these should all be zero or else there is duplication

5-element Vector{Any}:
 0
 0
 0
 0
 0

In [24]:
function ast2cont(x)
    d1[x]
end

ast2cont (generic function with 1 method)

In [25]:
m3=transform(m2,:ast => ByRow(ast2cont) => :cont)

,ast,studentID,mark,cont
,String7,Int64,Float64,Float64
1,mnll,104689,89.0,0.25
2,mnll,114185,92.0,0.25
3,mnll,116679,90.0,0.25
4,mnll,116824,89.0,0.25
5,mnll,162638,89.0,0.25
6,mnll,205269,92.0,0.25
7,mnll,224062,89.0,0.25
8,mnll,251559,90.0,0.25
9,mnll,252873,90.0,0.25


In [26]:
m3g=groupby(m3,:studentID)

,ast,studentID,mark,cont
,String7,Int64,Float64,Float64
1,mnll,104689,89.0,0.25
2,nvxk,104689,67.3,0.25
3,bemh,104689,80.0,0.25
4,tcbq,104689,100.0,0.05
5,zijh,104689,65.6,0.2
,ast,studentID,mark,cont
,String7,Int64,Float64,Float64
1,tcbq,995966,0.0,0.05


In [32]:
combine(m3g,[:mark,:cont] => ((m,c) -> dot(m,c)) => :total)

,studentID,total
,Int64,Float64
1,104689,77.195
2,114185,56.575
3,116679,79.145
4,116824,82.68
5,162638,77.97
6,205269,56.32
7,224062,67.695
8,251559,84.395
9,252873,81.16


In [33]:
# next steps: replace sdc by name and sno then sort and export to csv